# Linear Regression Again (this time it's ML flavour)

We'll redo our linear regression and this time we'll make it more "machine-learning-y". Let's first reload all the data (etc.):

In [1]:
# !pip install scikit-learn or !pip install --user scikit-learn

import numpy as np
import matplotlib.pyplot as plt 
import pandas as pd  
import seaborn as sns 
%matplotlib inline  

from sklearn.datasets import load_boston
boston_dataset = load_boston()

boston = pd.DataFrame(boston_dataset.data, columns=boston_dataset.feature_names)
boston_Y = boston_dataset.target

C:\Python\Python310\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function load_boston is deprecated; `load_boston` is deprecated in 1.0 and will be removed in 1.2.

    The Boston housing prices dataset has an ethical problem. You can refer to
    the documentation of this function for further details.

    The scikit-learn maintainers therefore strongly discourage the use of this
    dataset unless the purpose of the code is to study and educate about
    ethical issues in data science and machine learning.

    In this special case, you can fetch the dataset from the original
    source::

        import pandas as pd
        import numpy as np


        data_url = "http://lib.stat.cmu.edu/datasets/boston"
        raw_df = pd.read_csv(data_url, sep="\s+", skiprows=22, header=None)
        data = np.hstack([raw_df.values[::2, :], raw_df.values[1::2, :2]])
        target = raw_df.values[1::2, 2]

    Alternative datasets include the California housing dataset (i.e.


Actually, arguably, all we need to do is via splitting our data into training and test:

In [3]:
# split data into training and test
from sklearn.model_selection  import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(boston, boston_Y, test_size = 0.2)

# print the shapes to check everything is OK
print(X_train.shape)
print(X_test.shape)
print(Y_train.shape)
print(Y_test.shape)

(404, 13)
(102, 13)
(404,)
(102,)


The "shapes" shown here are number of rows by number of columns ... i.e. the shape of "X_train" is 404 rows by 13 columns. We'd expect the 13 columns (13 X's), and we can quickly do some math on the 506 row numbers ... $ 506 \hspace{0.2cm} / \hspace{0.2cm} 5 = 101.2 $ ... to see that 404 is rougly 80 % of the data (our "test_size" was 20% so our training data should be 80%) and that this all looks correct. The Y "shapes" show only rows but this is because we have only 1x value (1x column).

Using these subsets we can train the Linear Regression model on "X_train" and "Y_train", and then use this model to predict "X_test". If we can compare the predictions on "X_test" with the real values of "Y_test" this gives us a measure of how well our computer has learned to predict house prices:

In [4]:
from sklearn.linear_model import LinearRegression
lin_model = LinearRegression()

# fit the model to the training data
lin_model_fit = lin_model.fit(X_train, Y_train)

# predict the data
boston_predict = lin_model_fit.predict(X_test)

# calculate RMSE (root mean square error) and R^2 (predictive power)
from sklearn.metrics import mean_squared_error, r2_score
rmse = (np.sqrt(mean_squared_error(Y_test, boston_predict)))
r2 = r2_score(Y_test, boston_predict)

# print the performance metrics
print("Model performance")
print("--------------------------------------")
print('RMSE is {}'.format(rmse))
print('R2 score is {}'.format(r2))
print("\n")

Model performance
--------------------------------------
RMSE is 4.517076436928825
R2 score is 0.6985667852793457




Compared to the previous Notebook our RMSE is lower and $ R^2 $ higher (i.e. better in both cases)! This is a little suprising but probably just chance on how the data was split (random error means the metrics are superior). However, we can conclude that this hasn't made our model worse.

What it has done, however, is meant we have a model that is tested against its ability to make predictions on unseen data ... i.e. that it has learned about the process of predicting house prices and can do this well.

## L1 Regularisation (LASSO)

$ L1 $ Regression is performed in much the same way as Linear Regression. We have one other value (a hyperparameter - more on these later) of $ a $ which determines how much influence the $ L1 $ penalty has on how the line is fit in the model. We will dodge the issue of what this should be for now and just randomly set it as 0.5 

In [5]:
from sklearn.linear_model import Lasso
l1_model = Lasso(alpha=0.5)

# fit the model to the training data
l1_model_fit = l1_model.fit(X_train, Y_train)

# predict the data
boston_predict = l1_model_fit.predict(X_test)

# calculate RMSE (root mean square error) and R^2 (predictive power)
from sklearn.metrics import mean_squared_error, r2_score
rmse = (np.sqrt(mean_squared_error(Y_test, boston_predict)))
r2 = r2_score(Y_test, boston_predict)

# print the performance metrics
print("Model performance")
print("--------------------------------------")
print('RMSE is {}'.format(rmse))
print('R2 score is {}'.format(r2))
print("\n")

Model performance
--------------------------------------
RMSE is 4.665022866200911
R2 score is 0.6784979254792699




Performance is slightly lower than for our normal linear model. __However...__ this is somewhat to be expected and actually performance is only slightly worse. This may be a small price to pay for ensuring that our model is a little more robust to changing data.

Where we should see a much more fundamental difference is in the beta-weights/coefficients of the two models (the $ b $ values). Let's compare the two:

In [6]:
# print the beta values of the model (co-efficients)
betas = lin_model_fit.coef_
counter = 0
for col in boston.columns:
    if counter == 0:
        print("Beta weights/co-efficients - UNREGULARISED")
        print("-----------------------------------------")
    print(col + ": " + str(round(betas[counter], 4)))
    counter +=1
    
print("\n")

# print the beta values of the model (co-efficients)
betas_l1 = l1_model_fit.coef_
counter = 0
for col in boston.columns:
    if counter == 0:
        print("Beta weights/co-efficients - LASSO")
        print("-----------------------------------------")
    print(col + ": " + str(round(betas_l1[counter], 4)))
    counter +=1

Beta weights/co-efficients - UNREGULARISED
-----------------------------------------
CRIM: -0.121
ZN: 0.0569
INDUS: 0.0198
CHAS: 2.3179
NOX: -18.1873
RM: 3.807
AGE: 0.0
DIS: -1.642
RAD: 0.3
TAX: -0.0112
PTRATIO: -0.9878
B: 0.0098
LSTAT: -0.5462


Beta weights/co-efficients - LASSO
-----------------------------------------
CRIM: -0.098
ZN: 0.0603
INDUS: -0.0042
CHAS: 0.0
NOX: -0.0
RM: 2.5633
AGE: 0.0
DIS: -1.0953
RAD: 0.2659
TAX: -0.0144
PTRATIO: -0.8029
B: 0.0095
LSTAT: -0.667


As we can see some of the beta weights are now 0 ("INDUS", "CHAS" and "NOX" - of which "CHAS" and "NOX" were fairly large weights in the original model). We have effectively removed these features (X's) from the model - they are now (for instance) $ 0 * INDUS $ which will obviously be zero and therefore won't change the calculation of $ Y $. We have performed _feature selection_ - removing X's from the model.

We have also minimised the impact (weight) of all the X's. For instance, "RM" had a beta of 3.77 in the original model and only 0.74 in the LASSO model.

Overall we have made the model less reliant on certain features and removed others entirely (reducing model complexity). Both these elements mean our model is more robust/protected against over-fitting ... and the cost is only 0.05 in terms of $ R^2 $ score. 

## L2 Regularisation (Ridge)

L2 regularisation is performed in a very similar way in _scikit-learn_ ... and again we will set the hyperparameter $ a $ to the arbitrary value of 0.5:

In [7]:
from sklearn.linear_model import Ridge
l2_model = Ridge(alpha=0.5)

# fit the model to the training data
l2_model_fit = l2_model.fit(X_train, Y_train)

# predict the data
boston_predict = l2_model_fit.predict(X_test)

# calculate RMSE (root mean square error) and R^2 (predictive power)
from sklearn.metrics import mean_squared_error, r2_score
rmse = (np.sqrt(mean_squared_error(Y_test, boston_predict)))
r2 = r2_score(Y_test, boston_predict)

# print the performance metrics
print("Model performance")
print("--------------------------------------")
print('RMSE is {}'.format(rmse))
print('R2 score is {}'.format(r2))
print("\n")

Model performance
--------------------------------------
RMSE is 4.5205862598769695
R2 score is 0.698098168792698




Here we have model performance that slightly improves upon the original model ... while also adding regularisation protection. Let's compare the three beta weights:

In [8]:
# print the beta values of the model (co-efficients)
betas = lin_model_fit.coef_
counter = 0
for col in boston.columns:
    if counter == 0:
        print("Beta weights/co-efficients - UNREGULARISED")
        print("-----------------------------------------")
    print(col + ": " + str(round(betas[counter], 4)))
    counter +=1
    
print("\n")

# print the beta values of the model (co-efficients)
betas_l1 = l1_model_fit.coef_
counter = 0
for col in boston.columns:
    if counter == 0:
        print("Beta weights/co-efficients - LASSO")
        print("-----------------------------------------")
    print(col + ": " + str(round(betas_l1[counter], 4)))
    counter +=1
    
print("\n")

# print the beta values of the model (co-efficients)
betas_l2 = l2_model_fit.coef_
counter = 0
for col in boston.columns:
    if counter == 0:
        print("Beta weights/co-efficients - RIDGE")
        print("-----------------------------------------")
    print(col + ": " + str(round(betas_l2[counter], 4)))
    counter +=1

Beta weights/co-efficients - UNREGULARISED
-----------------------------------------
CRIM: -0.121
ZN: 0.0569
INDUS: 0.0198
CHAS: 2.3179
NOX: -18.1873
RM: 3.807
AGE: 0.0
DIS: -1.642
RAD: 0.3
TAX: -0.0112
PTRATIO: -0.9878
B: 0.0098
LSTAT: -0.5462


Beta weights/co-efficients - LASSO
-----------------------------------------
CRIM: -0.098
ZN: 0.0603
INDUS: -0.0042
CHAS: 0.0
NOX: -0.0
RM: 2.5633
AGE: 0.0
DIS: -1.0953
RAD: 0.2659
TAX: -0.0144
PTRATIO: -0.8029
B: 0.0095
LSTAT: -0.667


Beta weights/co-efficients - RIDGE
-----------------------------------------
CRIM: -0.1184
ZN: 0.0576
INDUS: -0.0031
CHAS: 2.2508
NOX: -12.7986
RM: 3.8257
AGE: -0.0049
DIS: -1.5645
RAD: 0.2875
TAX: -0.0116
PTRATIO: -0.9325
B: 0.01
LSTAT: -0.5526


As with L1/LASSO regression all our beta values are lower than in the original model - and therefore each feature is less influential. However, none have been reduced to zero. L2 cannot perform _feature selection_ (deleting X's) unlike with L1.

So better performance and regularisation added - seems like a good deal right? However, before we call it a day we have one other regularisation method we may try. 

## ElasticNet

ElasticNet uses both L1 and L2 regularisation. We now have an additonal value to set (hyperparameter - still going to come back to these) the _l1\_ratio._ As the name suggests the _l1\_ratio_ determines the proportion that is L1 versus L2 so "1" would be just L1 and "0" would just be L2. We'll arbitrarily go with 0.5 again. 

In [9]:
from sklearn.linear_model import ElasticNet
enet_model = ElasticNet(alpha=0.5, l1_ratio=0.5)

# fit the model to the training data
enet_model_fit = enet_model.fit(X_train, Y_train)

# predict the data
boston_predict = enet_model_fit.predict(X_test)

# calculate RMSE (root mean square error) and R^2 (predictive power)
from sklearn.metrics import mean_squared_error, r2_score
rmse = (np.sqrt(mean_squared_error(Y_test, boston_predict)))
r2 = r2_score(Y_test, boston_predict)

# print the performance metrics
print("Model performance")
print("--------------------------------------")
print('RMSE is {}'.format(rmse))
print('R2 score is {}'.format(r2))
print("\n")

Model performance
--------------------------------------
RMSE is 4.733687229395052
R2 score is 0.6689639094766657




Unsurprisingly the result is somewhere between the results of L1 and L2. Let's also check the weights:

In [10]:
# print the beta values of the model (co-efficients)
betas = lin_model_fit.coef_
counter = 0
for col in boston.columns:
    if counter == 0:
        print("Beta weights/co-efficients - UNREGULARISED")
        print("-----------------------------------------")
    print(col + ": " + str(round(betas[counter], 4)))
    counter +=1
    
print("\n")

# print the beta values of the model (co-efficients)
betas_l1 = l1_model_fit.coef_
counter = 0
for col in boston.columns:
    if counter == 0:
        print("Beta weights/co-efficients - LASSO")
        print("-----------------------------------------")
    print(col + ": " + str(round(betas_l1[counter], 4)))
    counter +=1
    
print("\n")

# print the beta values of the model (co-efficients)
betas_l2 = l2_model_fit.coef_
counter = 0
for col in boston.columns:
    if counter == 0:
        print("Beta weights/co-efficients - RIDGE")
        print("-----------------------------------------")
    print(col + ": " + str(round(betas_l2[counter], 4)))
    counter +=1
    
print("\n")

# print the beta values of the model (co-efficients)
betas_enet = enet_model_fit.coef_
counter = 0
for col in boston.columns:
    if counter == 0:
        print("Beta weights/co-efficients - ELASTICNET")
        print("-----------------------------------------")
    print(col + ": " + str(round(betas_enet[counter], 4)))
    counter +=1

Beta weights/co-efficients - UNREGULARISED
-----------------------------------------
CRIM: -0.121
ZN: 0.0569
INDUS: 0.0198
CHAS: 2.3179
NOX: -18.1873
RM: 3.807
AGE: 0.0
DIS: -1.642
RAD: 0.3
TAX: -0.0112
PTRATIO: -0.9878
B: 0.0098
LSTAT: -0.5462


Beta weights/co-efficients - LASSO
-----------------------------------------
CRIM: -0.098
ZN: 0.0603
INDUS: -0.0042
CHAS: 0.0
NOX: -0.0
RM: 2.5633
AGE: 0.0
DIS: -1.0953
RAD: 0.2659
TAX: -0.0144
PTRATIO: -0.8029
B: 0.0095
LSTAT: -0.667


Beta weights/co-efficients - RIDGE
-----------------------------------------
CRIM: -0.1184
ZN: 0.0576
INDUS: -0.0031
CHAS: 2.2508
NOX: -12.7986
RM: 3.8257
AGE: -0.0049
DIS: -1.5645
RAD: 0.2875
TAX: -0.0116
PTRATIO: -0.9325
B: 0.01
LSTAT: -0.5526


Beta weights/co-efficients - ELASTICNET
-----------------------------------------
CRIM: -0.1063
ZN: 0.0648
INDUS: -0.0274
CHAS: 0.0
NOX: -0.0
RM: 1.7902
AGE: 0.0065
DIS: -1.1318
RAD: 0.299
TAX: -0.0153
PTRATIO: -0.8545
B: 0.0089
LSTAT: -0.7182


Again, as we may expect, the results are somewhere between the two. "CHAS" and "NOX" have betas of zero (have been removed) but not "INDUS". All other beta weights are reduced to somewhere between the L1 and L2 models.

## BONUS! Which is best?

Hahahahaha ... you know what I'm going to say:




### IT DEPENDS!

If you are not worried about overfitting use the original model (but if you don't care about overfitting are you doing ML?). If you have a lot of features and want some removed you may use L1 or Elastic Net. If you care most about performance use the L2 model. In this case I'd probably go with L2 as there aren't a lot of features and the performance is best.